# Algorithmic Foundations: Logistic Regression & Vectorization

## 1. Project Goal
The objective of this notebook is to implement **Logistic Regression from scratch** to understand the mathematical optimization behind the algorithm. 

We will focus on:
1. **Mathematical Implementation:** Coding the Sigmoid activation and Cost function manually.
2. **Vectorization:** replacing `for-loops` with Linear Algebra (Matrix operations) using NumPy.
3. **Performance Analysis:** Benchmarking the training speed difference between iterative and vectorized approaches.

## 2. Mathematical Background
The core of Logistic Regression is minimizing the Cost Function $J(w,b)$ using **Gradient Descent**:
$$w = w - \alpha \frac{\partial J}{\partial w}$$

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import time 

# 1. Generate Synthetic Data
np.random.seed(1)
m = 10000  # Number of examples (Large enough to see speed difference)
n = 500    # Number of features

# Create random feature matrix X and target vector y
X = np.random.rand(m, n)
y = np.random.randint(0, 2, (m, 1)) # Binary target (0 or 1)

# Initialize parameters (weights w and bias b)
w_init = np.random.rand(n, 1)
b_init = 0.5

print(f"Dataset generated: {m} examples with {n} features each.")
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")

Dataset generated: 10000 examples with 500 features each.
Shape of X: (10000, 500), Shape of y: (10000, 1)


In [7]:
def sigmoid(z):
    """
    Compute the sigmoid of z
    """
    return 1 / (1 + np.exp(-z))

# Quick test
print(f"Sigmoid of 0: {sigmoid(0)}") # Should be 0.5

Sigmoid of 0: 0.5


In [8]:
# --- Version 1: Unoptimized Loop (Iterative) ---
def compute_gradient_loop(X, y, w, b):
    m, n = X.shape
    dj_dw = np.zeros((n, 1))
    dj_db = 0.

    for i in range(m):
        # Prediction for single example i
        z_wb = 0
        for j in range(n): 
            z_wb += X[i, j] * w[j]
        z_wb += b
        f_wb = sigmoid(z_wb)

        # Error calculation
        err = f_wb - y[i]
        
        # Accumulate gradients
        dj_db += err
        for j in range(n):
            dj_dw[j] += err * X[i, j]
            
    dj_dw = dj_dw / m
    dj_db = dj_db / m
        
    return dj_db, dj_dw

# --- Version 2: Optimized Vectorization (Matrix Multiplication) ---
def compute_gradient_matrix(X, y, w, b):
    m, n = X.shape
    
    # 1. Calculate predictions for ALL examples at once
    z = np.dot(X, w) + b     # Matrix Product
    f_wb = sigmoid(z)        # Vectorized Sigmoid
    
    # 2. Calculate Error
    err = f_wb - y
    
    # 3. Calculate Gradients using Transpose
    dj_dw = (1/m) * np.dot(X.T, err)
    dj_db = (1/m) * np.sum(err)
    
    return dj_db, dj_dw

In [9]:
print("--- Starting Performance Benchmark ---\n")

# Test 1: Loop Version
start_time = time.time()
dj_db_loop, dj_dw_loop = compute_gradient_loop(X, y, w_init, b_init)
end_time = time.time()
time_loop = end_time - start_time
print(f"Loop Version Time:      {time_loop:.4f} seconds")

# Test 2: Vectorized Version
start_time = time.time()
dj_db_vec, dj_dw_vec = compute_gradient_matrix(X, y, w_init, b_init)
end_time = time.time()
time_vec = end_time - start_time
print(f"Vectorized Version Time: {time_vec:.4f} seconds")

# Calculate Speedup
speedup = time_loop / time_vec
print(f"\nResult: Vectorization is {speedup:.1f}x faster!")

--- Starting Performance Benchmark ---

Loop Version Time:      7.3291 seconds
Vectorized Version Time: 0.0037 seconds

Result: Vectorization is 1970.3x faster!


In [10]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    w = w_in
    b = b_in
    J_history = []
    
    for i in range(num_iters):
        # Use the optimized matrix function
        dj_db, dj_dw = compute_gradient_matrix(X, y, w, b)
        
        # Update parameters
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        # Record cost (optional implementation for brevity)
        if i % 100 == 0:
             print(f"Iteration {i}: Parameters updating...")
             
    return w, b

# Run Training
print("Training final model...")
w_final, b_final = gradient_descent(X, y, w_init, b_init, alpha=0.1, num_iters=1000)
print("Training Complete.")

Training final model...
Iteration 0: Parameters updating...
Iteration 100: Parameters updating...
Iteration 200: Parameters updating...
Iteration 300: Parameters updating...
Iteration 400: Parameters updating...
Iteration 500: Parameters updating...
Iteration 600: Parameters updating...
Iteration 700: Parameters updating...
Iteration 800: Parameters updating...
Iteration 900: Parameters updating...
Training Complete.


## 3. Conclusion & Key Takeaways

1. **Computational Efficiency:** The vectorized implementation is orders of magnitude faster (often >500x) than the loop-based approach. This is crucial when working with large datasets (Big Data).
2. **Scalability:** By using Linear Algebra (`np.dot`), we leverage modern CPU/GPU architecture, allowing us to train models on millions of rows in seconds.
3. **Core Understanding:** Implementing the gradient update rule manually ensures a deep understanding of how neural networks learn via backpropagation.